<a href="https://colab.research.google.com/github/RomulusGwelt/deep-learning-kbtu-2019/blob/master/DogBreedXception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading data 

Note: You need to upload kaggle API token for the following cell





In [0]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json
# Next, install the Kaggle API client.
!pip install -q kaggle

# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

#download the dataset for the dog-breed identification challenge https://www.kaggle.com/c/dog-breed-identification
!kaggle competitions download -c dog-breed-identification

#unzip the downloaded files
!unzip test.zip
!unzip train.zip
!unzip labels.csv.zip
!mkdir models

Cyclic LR is unfortunately not a part of keras.

CyclicLR.py can be found at my google drive or https://github.com/bckenstler/CLR

In [0]:

# !pip uninstall tensorflow
# !pip install tensorflow-gpu

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from keras_preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg

import numpy as np 
import pandas as pd 

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
import keras
from math import ceil
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.inception_v3 import preprocess_input
# from keras.applications.resnet50 import preprocess_input
from keras.applications.xception import preprocess_input
import os 
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras.callbacks import EarlyStopping, ModelCheckpoint
from cyclicLR import CyclicLR
from keras.optimizers import Adam, SGD, RMSprop
import datetime
from keras.applications import xception
# print(device_lib.list_local_devices())
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

In [0]:
print(tf.__version__)

1.15.0


# Pre-processing data


In [0]:
ds = pd.read_csv('labels.csv')
ds['image'] = ds.apply(lambda row: str(row['id']) + ".jpg", axis=1)
ds = ds.sample(frac=1).reset_index(drop=True)
ds.head(10)

print(ds.head())


                                 id  ...                                 image
0  9ce66abbf81462439d6dd6805af1d9a1  ...  9ce66abbf81462439d6dd6805af1d9a1.jpg
1  72ef6af03c9603ce6ae39475861fdd64  ...  72ef6af03c9603ce6ae39475861fdd64.jpg
2  fa5054c5187c7171c546bd6a46b8346f  ...  fa5054c5187c7171c546bd6a46b8346f.jpg
3  c0f1af3542270b1e4f1be3403d17ec35  ...  c0f1af3542270b1e4f1be3403d17ec35.jpg
4  168e2da635938b82819b8a45bbd3dd0d  ...  168e2da635938b82819b8a45bbd3dd0d.jpg

[5 rows x 3 columns]


In [0]:
batch_size = 16

In [0]:
# image_generator = ImageDataGenerator(
#     # rescale=1./255.,
#     preprocessing_function=preprocess_input,
#     rotation_range=25,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest',
#     validation_split=0.1
# )
image_generator= ImageDataGenerator(
      preprocessing_function=preprocess_input, 
      # rescale=1./255.,
      horizontal_flip=True,
      width_shift_range = 0.2,
      height_shift_range = 0.2,
      validation_split=0.2)

In [0]:
training_generator = image_generator.flow_from_dataframe(
      dataframe=ds,
      directory="train",
      # directory="/content/gdrive/My Drive/Colab Notebooks/train",
      x_col="image",
      y_col="breed",
      target_size=(299,299),
      batch_size=batch_size,
      subset="training"
)

validation_generator = image_generator.flow_from_dataframe(
      dataframe=ds,
      directory="train",
      # directory="/content/gdrive/My Drive/Colab Notebooks/train",
      x_col="image",
      y_col="breed",
      target_size=(299,299),
      batch_size=batch_size,
      subset="validation"
)

Found 8178 validated image filenames belonging to 120 classes.
Found 2044 validated image filenames belonging to 120 classes.


In [0]:
input_shape = (299, 299, 3)

classes = len(training_generator.class_indices)

# Creating & tweaking a model

Creating an Xception model with 1 dense layer (1024) with a dropout of 0.25

In [0]:
# base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
# base_model = ResNet50(input_shape=input_shape, include_top=False, weights='imagenet')
# base_model = InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')
base_model = xception.Xception(input_shape=input_shape, weights='imagenet', include_top=False)
# model = xception.Xception(input_shape=input_shape, weights=None, include_top=True, pooling='avg',classes = classes)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
predictions = Dense(classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

Creating an early stopping, so that our model would stop running after X epochs of the stagnation

In [0]:
# checkpoints
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
STAMP = "{}_dog_breed_model".format(datetime.date.today().strftime("%Y-%m-%d"))

bst_model_path = "models/{}.h5".format(STAMP)
model_checkpoint = ModelCheckpoint(bst_model_path,
                                   save_best_only=True,
                                   save_weights_only=True)

# Authors suggest setting step_size = (2-8) x (training iterations in epoch)
step_size = 2000
clr = CyclicLR(base_lr=0.0001,
               max_lr=0.001,
               step_size=step_size,
               mode='exp_range',
               gamma=0.99994)

Compiling model with RMSprop optimizer


In [0]:
for layer in base_model.layers:
    layer.trainable = False

optimizer = RMSprop(lr=0.001, rho=0.9)
# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

model.compile(optimizer=optimizer,
              # loss='sparse_categorical_crossentropy',
              loss = keras.losses.categorical_crossentropy,
              metrics=["accuracy"])

**Start training the model**

In [71]:
hist = model.fit_generator(
                           generator=training_generator,
                           steps_per_epoch=ceil(0.8 * (ds.size / batch_size)),
                           
                           validation_data=validation_generator,
                           validation_steps=ceil(0.2 * (ds.size / batch_size)),
                           
                           epochs=50, 
                           callbacks=[early_stopping, model_checkpoint, clr],
)
# model.fit_generator(
#     generator=training_generator,
#     steps_per_epoch=ceil(0.8 * (ds.size / batch_size)),

#     validation_data=validation_generator,
#     validation_steps=ceil(0.2 * (ds.size / batch_size)),

#     epochs=50,
#     verbose=1
# ) 

Epoch 1/50
   4/1534 [..............................] - ETA: 23:33 - loss: 4.8422 - acc: 0.0000e+00

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.450467). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1534/1534 [==============================] - 566s 369ms/step - loss: 1.2490 - acc: 0.7088 - val_loss: 0.4829 - val_acc: 0.8457
Epoch 2/50
1534/1534 [==============================] - 554s 361ms/step - loss: 0.6240 - acc: 0.8244 - val_loss: 0.4592 - val_acc: 0.8648
Epoch 3/50
1534/1534 [==============================] - 554s 361ms/step - loss: 0.3867 - acc: 0.8827 - val_loss: 0.4362 - val_acc: 0.8640
Epoch 4/50
1534/1534 [==============================] - 554s 361ms/step - loss: 0.4351 - acc: 0.8732 - val_loss: 0.4663 - val_acc: 0.8615
Epoch 5/50
1534/1534 [==============================] - 558s 364ms/step - loss: 0.3771 - acc: 0.8914 - val_loss: 0.4577 - val_acc: 0.8655
Epoch 6/50
1534/1534 [==============================] - 558s 364ms/step - loss: 0.2840 - acc: 0.9102 - val_loss: 0.5609 - val_acc: 0.8563
Epoch 7/50
1534/1534 [==============================] - 560s 365ms/step - loss: 0.3648 - acc: 0.8968 - val_loss: 0.5512 - val_acc: 0.8555
Epoch 8/50
1534/1534 [=======================

# Saving the model

In [0]:
model.save('model.h5')


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [74]:
uploaded = drive.CreateFile({'title': 'model.h5'})
uploaded.SetContentFile('model.h5')
uploaded.Upload()
# uploaded.Download()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 18Z9yq6XYSH5c2vMeuSU1RHB1zwIcuWNF


# Predicting outputs for test dataset

Evaluating our model on validation(just to be sure)

In [75]:
model.evaluate_generator(generator=validation_generator)

[0.5854470241238926, 0.8571428571428571]

Unzipping sample csv

In [76]:
!unzip sample_submission.csv.zip

Archive:  sample_submission.csv.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Adding "image" column to the csv that represents the path to that image

In [77]:
testds=pd.read_csv('sample_submission.csv')

testds['image'] = testds.apply(lambda row: str(row['id']) + ".jpg", axis=1)
testds = testds.sample(frac=1).reset_index(drop=True)

test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)


print(testds.head())
print(testds['id'])

test_generator = test_datagen.flow_from_dataframe(
    dataframe=testds,
    directory="test",
    x_col='image',
    y_col=None,
    target_size=(299,299),
    batch_size=1,
    class_mode=None,
)

                                 id  ...                                 image
0  26011b75add6ef201dcc91b41ae9a329  ...  26011b75add6ef201dcc91b41ae9a329.jpg
1  16cd0ecab645b8d0629296fe0ee728f0  ...  16cd0ecab645b8d0629296fe0ee728f0.jpg
2  f7e3178f426d312a5a06ee7e012efbec  ...  f7e3178f426d312a5a06ee7e012efbec.jpg
3  a87389ccf3e82e66fcbab9426d4f50ab  ...  a87389ccf3e82e66fcbab9426d4f50ab.jpg
4  f3cbc8bcd54963296f75377d6fe931d3  ...  f3cbc8bcd54963296f75377d6fe931d3.jpg

[5 rows x 122 columns]
0        26011b75add6ef201dcc91b41ae9a329
1        16cd0ecab645b8d0629296fe0ee728f0
2        f7e3178f426d312a5a06ee7e012efbec
3        a87389ccf3e82e66fcbab9426d4f50ab
4        f3cbc8bcd54963296f75377d6fe931d3
                       ...               
10352    c879dc8583ff90b3ceb67f1206e4c572
10353    d6d1131779e16248dcc207f4997b2930
10354    5c46331b5091e6152a7949485c8840e7
10355    cc93dc31b755472bec8464e4fa33cc3d
10356    101957d48b542794672e84eeae2f6595
Name: id, Length: 10357, dtype: object
F

Run the model on test data

In [78]:
test_generator.reset()
pred=model.predict_generator(test_generator,verbose=1)

10357/10357 [==============================] - 164s 16ms/step


Creating an output csv

In [79]:
labels = (training_generator.class_indices)
labels = list(labels.keys())
df = pd.DataFrame(data=pred,
                 columns=labels)

columns = list(df)
columns.sort()
df = df.reindex(columns=columns)

filenames = testds["id"]
df["id"]  = filenames

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,26011b75add6ef201dcc91b41ae9a329,7.826669e-12,1.645849e-12,1.765954e-12,4.480172e-10,7.578008e-10,1.408454e-09,5.633014e-13,3.199229e-13,9.997924e-01,2.005383e-05,1.069342e-12,8.176647e-08,1.110525e-10,6.311434e-09,8.010481e-09,2.043302e-08,5.559395e-10,3.264209e-12,5.905691e-13,1.039725e-11,5.467148e-12,2.002303e-11,1.597397e-13,1.591856e-12,3.800524e-07,1.375628e-09,1.683802e-10,1.126989e-10,1.011195e-09,2.627461e-13,1.747168e-09,5.866502e-08,1.185427e-08,9.085994e-11,5.678735e-12,1.199028e-11,4.661217e-11,2.237534e-11,7.987789e-14,...,2.253096e-12,3.455830e-12,5.746121e-09,3.375559e-11,4.201042e-13,2.068750e-11,3.549071e-11,3.502745e-13,5.622672e-11,4.506790e-11,2.081803e-12,5.104230e-12,7.063652e-07,8.855187e-12,5.000700e-10,2.106841e-12,1.006499e-11,1.761435e-10,2.897728e-11,2.616302e-11,3.543702e-10,1.353768e-10,4.075368e-11,7.142895e-11,1.715132e-11,1.136015e-09,6.381316e-12,3.634633e-12,3.564880e-10,5.902145e-09,7.298375e-12,9.654676e-16,7.746580e-14,5.478250e-06,2.830257e-11,1.795497e-04,1.155821e-10,4.517565e-11,1.573942e-11,1.885674e-11
1,16cd0ecab645b8d0629296fe0ee728f0,1.037244e-09,4.018015e-11,6.582867e-12,6.607528e-10,6.720940e-08,6.239685e-07,2.538525e-09,2.544962e-10,1.690079e-11,6.527962e-09,1.144953e-10,1.729027e-07,1.798264e-10,2.725477e-09,2.700863e-10,1.811146e-09,9.596426e-01,6.111384e-10,1.710388e-08,6.236200e-10,4.069722e-11,1.515150e-11,2.340766e-10,8.095709e-11,1.102697e-09,1.821858e-10,7.302321e-11,4.273624e-07,4.087309e-08,1.472894e-10,4.779327e-10,3.361457e-11,2.965434e-11,4.033021e-02,2.263445e-07,1.016876e-09,4.878202e-11,1.839021e-10,2.654096e-10,...,2.592239e-09,2.205052e-11,2.021598e-08,2.221951e-09,2.164199e-06,1.736328e-10,6.581896e-09,3.038296e-10,6.992799e-10,1.875901e-09,5.518020e-09,1.066896e-10,2.627053e-08,5.606236e-11,1.228279e-06,3.708089e-08,5.055988e-12,1.788864e-09,1.142646e-09,1.651897e-05,3.076752e-09,1.936617e-08,1.377710e-09,7.920455e-12,2.462894e-08,6.076402e-09,1.485348e-09,1.965747e-10,2.154122e-09,2.454217e-09,8.405535e-10,2.970259e-10,2.624035e-10,3.376754e-09,4.222491e-10,4.916017e-09,1.204327e-08,9.559521e-09,2.860146e-09,2.303801e-09
2,f7e3178f426d312a5a06ee7e012efbec,1.484290e-13,1.279633e-17,5.069432e-17,7.540204e-13,1.675428e-13,2.582845e-13,8.626439e-13,3.309892e-13,4.171074e-14,2.199219e-13,1.165133e-16,2.776754e-14,1.229604e-14,1.737514e-16,3.292234e-15,1.148135e-13,1.066602e-11,5.694096e-14,2.296462e-18,8.323844e-17,2.539969e-15,1.999707e-15,9.367291e-16,1.212896e-18,1.033599e-14,8.609044e-14,4.982994e-12,2.880437e-14,1.673900e-14,1.818491e-15,9.999999e-01,2.952626e-15,1.328181e-14,1.070295e-11,2.905563e-15,1.350332e-14,4.364183e-14,7.780785e-10,1.811512e-15,...,9.882826e-14,4.229307e-10,6.426185e-17,1.009331e-16,6.606369e-16,9.042422e-15,4.079000e-10,5.408045e-10,8.367468e-15,3.124970e-12,1.241884e-13,9.665946e-13,5.820259e-15,4.362115e-16,2.765934e-10,2.373927e-12,1.135

In [0]:
df.to_csv("submission.csv",index=False)

Saving csv to a google drive

In [83]:
uploaded = drive.CreateFile({'title': 'submission.csv'})
uploaded.SetContentFile('submission.csv')
uploaded.Upload()
# uploaded.Download()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1cAdSyNNO4vGOhVPTtQDHhbdP5XM2aTvZ


In [84]:
uploaded = drive.CreateFile({'title': 'best.h5'})
uploaded.SetContentFile('/content/models/2019-11-18_dog_breed_model.h5')
uploaded.Upload()
# uploaded.Download()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1yUvCRBrEfB8_CadOk8z_9iPBA0KnFVdj
